# Steps for all of us

Choose a dataset that you want to use. 

You may do whatever steps you think necessary for building the best classifier.

Take the data you chose and do whatever massaging you think is necessary: standardizing, scaling, feature engineering/ transforming, feature selection, etc.  

Build a classifier however you see fit. You may want build one and tweak the paramters manually or use some sort of grid search to look through all possible parameters. 

Remember: The same model built on the massaged data may perform better than if the data was untouched. It may be more conveniant to chose a standared massaging pipeline and tweak a model to that data.

After you massage your data, follow these steps:

if you want to balance your target (which you should) follow along these lines:

### build your features data and target data

- X = df.drop(columns = "whatever your target name is")
- y = df"whatever your target name is"

### split the data into training and testing

- X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2019)

### create the oversampled data to train on 

- oversampler = SMOTE(random_state = 2019)
- X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

### Put the oversampled data back into a dataframe

- X_train_oversampled = pd.DataFrame(X_train_oversampled, columns = X_train.columns)
- y_train_oversampled = pd.Series(y_train_oversampled)

### Build your classifier here. As an example:

- xgb_clf = xgb.XGBClassifier(max_depth=5, n_estimators=100, colsample_bytree=0.3, learning_rate=0.1, n_jobs=-1)

 
### Fit to the oversampled data; this will train the classifier on the oversampled data

- xgb_clf.fit(X_train_oversampled, y_train_oversampled)

### Use 5-fold cross validation to see how well the classfier you built is doing on test data. 
Some points: you have to substitute your classifer name in the cross_val_score function 

- kfold = KFold(n_splits=5, random_state=2019)
- results = cross_val_score(xgb_clf, X_test, y_test, cv=kfold, scoring = 'f1')


## It may be best to keep all of your models you built; have a log of them to see their scores and keep a record of your process of building your data. 



In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from Modules import *
sns.set()
%matplotlib inline
import imblearn


### read in the full sequential data

In [5]:
df = pd.read_csv('Trimmed_sequential_data.csv')
y = df['Y']

df.head().T

,0,1,2,3,4
AGE,24.000000,26.000000,34.000000,37.000000,57.000000
PAY_1,2.000000,-1.000000,0.000000,0.000000,-1.000000
PAY_2,2.000000,2.000000,0.000000,0.000000,0.000000
PAY_3,-1.000000,0.000000,0.000000,0.000000,-1.000000
PAY_4,-1.000000,0.000000,0.000000,0.000000,0.000000
PAY_5,-2.000000,0.000000,0.000000,0.000000,0.000000
PAY_6,-2.000000,2.000000,0.000000,0.000000,0.000000
Y,1.000000,1.000000,0.000000,0.000000,0.000000
SEX_Female,1.000000,1.000000,1.000000,1.000000,0.000000
SEX_Male,0.000000,0.000000,0.000000,0.000000,1.000000


In [6]:
#trying binning the PAY_ columns to something more specific
#any value that is late more than 2 months will be "more_than 2_months"
#any value that is one month late will be "1_month_late"
#any value on time will be "on_time"
#any 0, or -2 value, which are not defined by dictionary, will be "other"
values = [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
columns = ['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

for col in columns:
  df[col] = df[col].replace([values[0], values[2]], 'Other')
  df[col] = df[col].replace(values[1], "on_time")
  df[col] = df[col].replace(values[3], "one_month_late")
  df[col] = df[col].replace(values[4:], "more_than_two_month_late")




In [7]:
df = pd.get_dummies(df)
df.head(2).T

,0,1
AGE,24.000000,26.000000
Y,1.000000,1.000000
SEX_Female,1.000000,1.000000
SEX_Male,0.000000,0.000000
EDUCATION_Graduate School,0.000000,0.000000
EDUCATION_Other,0.000000,0.000000
EDUCATION_University,1.000000,1.000000
MARRIAGE_Married,1.000000,0.000000
MARRIAGE_Non-married,0.000000,1.000000
PERCENT_OF_LIMIT_BAL1,0.195650,0.022350


In [9]:
#convert to csv
#df.to_csv('Final_trimmed_sequential_data.csv')

In [14]:
#convert only sequential data to csv
#df_seq_only = df.iloc[:,  9:]
#df_seq_full = pd.concat([df_seq_only, y], axis = 1)
#df_seq_full.to_csv('Sequential_data_only.csv')

In [8]:
evaluate_baseline(df, clf = 'Logistic')

10-fold f1 scores:
[0.48902821 0.49032258 0.54385965 0.58536585 0.51190476 0.54441261
 0.50555556 0.54487179 0.49700599 0.51006711]

corss-validation f1 score: 0.522239411849253


In [6]:
evaluate_baseline(df, clf = 'SVM')

10-fold f1 scores:
[0.44805195 0.48734177 0.51026393 0.55494505 0.51051051 0.54038997
 0.54545455 0.54434251 0.50720461 0.50657895]

corss-validation f1 score: 0.5155083798842262


In [5]:
evaluate_baseline(df, clf = 'RF')

10-fold f1 scores:
[0.40495868 0.50819672 0.4981685  0.52317881 0.41947566 0.48611111
 0.5037037  0.48995984 0.5037037  0.45528455]

corss-validation f1 score: 0.47927412712651324
